# Taxonomic assignment: Downstream analyses

In [ ]:
pip install openpyxl

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from openpyxl.workbook import Workbook

/home/lubo/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
# load dataset
marker="Full18S"
cell="cell1"
project_path="/home/lubo/code/wRajter/vampyrella_2023"
# at uni local machine:"/home/lubomir/projects/vampyrella_2023"
data_path=f"{project_path}/raw_data/qiime_output/dada2_output/asv_summary_table_{marker}_{cell}.tsv"


asv_data=pd.read_csv(data_path, sep="\t")

In [13]:
# save as an excel table
asv_data.to_excel(f'ASV_summary_table_{marker}_{cell}.xlsx')

In [ ]:
asv_data

In [ ]:
asv_data.info()

In [ ]:
# vampyrellida ASVs

vamp = asv_data[asv_data['Order'] == 'Vampyrellida']

vamp[['ID', 'Total', 'A3', 'Mock', 'NH1', 'NH4', 'Sim17', 'Sim22', 'Th16',
   'Th38', 'Th40', 'X17007', 'Order', 'Family', 'Genus', 'Species', 
   'Percent_similarity']]

In [ ]:
# count plot
%matplotlib inline
sns.set(rc={'figure.figsize':(15,6)})

sns.histplot(data=asv_data, x='Domain');

## Relative Abundance Barplots

In [ ]:
### Import needed modules
#ggplot is used for plotting and is based on the R package ggplot2
#Pandas is a widely-used Python library for statistics, particularly on tabular data. It installed with Anaconda.
#We will use it to read and write tables in .csv format.



In [ ]:
# prepare data

relab_data = asv_data[['ID',
                       'Total', 
                       'A3', 
                       'Mock', 
                       'NH1', 
                       'NH4', 
                       'Sim17', 
                       'Sim22', 
                       'Th16',
                       'Th38', 
                       'Th40', 
                       'X17007',
                      'Domain']]

In [ ]:
relab_data_melt = relab_data.melt(id_vars=['ID', 'Domain'], var_name='Sample', value_name='Abundance')

In [ ]:
%matplotlib inline
sns.set(rc={'figure.figsize':(15,6)})

relab_data.plot(kind ='bar', stacked = True)